In [10]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import time

In [11]:
# driver_check = webdriver.Chrome()
# driver_check.get("https://www.google.com")

In [ ]:
# # Optional: run headless (no browser window)
# options = Options()
# options.headless = True

# # Set up ChromeDriver
# driver = webdriver.Chrome(options=options)

# columns = np.array([ 
#     "temperature_max", "temperature_avg", "temperature_min",
#     "dew_point_max", "dew_point_avg", "dew_point_min",
#     "humidity_max", "humidity_avg", "humidity_min",
#     "wind_speed_max", "wind_speed_avg", "wind_speed_min",
#     "pressure_max", "pressure_avg", "pressure_min",
#     "precipitation",
# ])

# for y in range(1997, 2026)[::-1]:
#     record_count = 0
#     data = []
#     first = True
#     for m in range(1, 13)[::-1]:

#         url = f'https://www.wunderground.com/history/monthly/th/bangkok/VTBD/date/{y}-{m}'
#         print(f"Fetching data for {y}-{m}...")
#         driver.get(url)
#         time.sleep(5)  # Adjust as needed for content to load

#         try:
#             print(f"Extracting data for {y}-{m}...")
#             table = driver.find_element(By.CSS_SELECTOR, "table.days")
#             ttable = table.find_elements(By.CSS_SELECTOR, "table")

#             for i, t in enumerate(ttable):
#                 rows = t.find_elements(By.CSS_SELECTOR, "tr")
#                 for j, r in enumerate(rows):
#                     cells = r.find_elements(By.CSS_SELECTOR, "td")
#                     if i == 0:
#                         data.append([f'{c.text}-{m}-{y}' for c in cells])
#                     else:
#                         data[record_count + j].extend([c.text for c in cells])

#             record_count += len(rows)     
#             print(f'Successfully Extracted: {y}-{m}\n--------------------------------')
#         except Exception as e:
#             print(f"!!! Failed to extract {y}-{m}: {e}")

#     temp_df = pd.DataFrame(data)
#     temp_df = temp_df[temp_df[16] != 'Total']

#     df = pd.DataFrame(
#         data=temp_df.iloc[0:, 1:].values,
#         index=temp_df[0],
#         columns=columns,
#         dtype=np.float64,
#     )
#     df.index = pd.to_datetime(df.index, format='%d-%m-%Y')
#     df.index.name = 'date'

#     df.to_csv("../../data/a1/bangkok_weather.csv", mode='a', header=False, index=True)

# driver.quit()

In [ ]:
import numpy as np
import pandas as pd
import requests
import time
from datetime import datetime
import calendar

def fetch_monthly_data(year, month):
    """Fetch weather data for a specific year and month"""
    # Format dates for API request
    start_date = f"{year}{month:02d}01"
    
    # Get the last day of the month
    last_day = calendar.monthrange(year, month)[1]
    end_date = f"{year}{month:02d}{last_day:02d}"
    
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en,th;q=0.9,en-US;q=0.8',
        'cache-control': 'no-cache',
        'origin': 'https://www.wunderground.com',
        'pragma': 'no-cache',
        'priority': 'u=1, i',
        'referer': 'https://www.wunderground.com/',
        'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
    }
    
    params = {
        'apiKey': 'e1f10a1e78da46f5b10a1e78da96f525',
        'units': 'e',
        'startDate': start_date,
        'endDate': end_date,
    }
    
    print(f"Fetching data for {year}-{month:02d}...")
    try:
        response = requests.get(
            'https://api.weather.com/v1/location/VTBD:9:TH/observations/historical.json',
            params=params,
            headers=headers,
        )
        
        if response.status_code == 200:
            data = response.json()
            print(f"Successfully fetched {len(data.get('observations', []))} observations for {year}-{month:02d}")
            print(f"Parameters: {params}")
            print("--------------------------------")
            return data
        else:
            print(f"!!! Failed to fetch {year}-{month:02d}: HTTP {response.status_code}")
            print(f"Parameters: {params}")
            print("--------------------------------")
            return None
    except Exception as e:
        print(f"!!! Error fetching {year}-{month:02d}: {e}")
        return None

def process_historical_data(start_year=1997, end_year=2025):
    """Process historical weather data for the given range of years, preserving hourly records"""
    all_hourly_data = []
    
    # Iterate through years and months in reverse order (newest first)
    for year in range(end_year, start_year - 1, -1):
        for month in range(12, 0, -1):
            # Skip future dates
            current_date = datetime.now()
            if year > current_date.year or (year == current_date.year and month > current_date.month):
                continue
                
            monthly_data = fetch_monthly_data(year, month)
            
            if monthly_data and 'observations' in monthly_data:
                # Process all hourly observations
                for obs in monthly_data['observations']:
                    # Convert timestamp to datetime with time
                    timestamp = datetime.fromtimestamp(obs['valid_time_gmt'])
                    
                    # Extract all available data fields
                    hourly_data = {
                        'timestamp': timestamp,
                        'temp': obs.get('temp'),
                        'dewPt': obs.get('dewPt'),
                        'rh': obs.get('rh'),
                        'wspd': obs.get('wspd'),
                        'gust': obs.get('gust'),
                        'pressure': obs.get('pressure'),
                        'precip_hrly': obs.get('precip_hrly'),
                        'precip_total': obs.get('precip_total'),
                        'snow_hrly': obs.get('snow_hrly'),
                        'wdir': obs.get('wdir'),
                        'wdir_cardinal': obs.get('wdir_cardinal'),
                        'clds': obs.get('clds'),
                        'vis': obs.get('vis'),
                        'feels_like': obs.get('feels_like'),
                        'uv_index': obs.get('uv_index'),
                        'wx_phrase': obs.get('wx_phrase'),
                        'day_ind': obs.get('day_ind'),
                        'max_temp': obs.get('max_temp'),
                        'min_temp': obs.get('min_temp'),
                        'heat_index': obs.get('heat_index'),
                        'wc': obs.get('wc')  # Wind chill
                    }
                    
                    all_hourly_data.append(hourly_data)
            
            # Add a small delay between requests to avoid rate limiting
            time.sleep(1)
    
    # Convert to DataFrame
    if all_hourly_data:
        df = pd.DataFrame(all_hourly_data)
        
        # Set timestamp as index but keep it as a column too
        df['date'] = df['timestamp']
        df.set_index('timestamp', inplace=True)
        
        # Sort by timestamp
        df.sort_index(inplace=True)
        
        # Save to CSV - raw hourly data
        df.to_csv("bangkok_weather_hourly.csv", mode='w', header=True, index=True)
        print(f"Saved {len(df)} hourly observations to CSV file")
        
        # Optionally create daily aggregated data similar to your original format
        daily_df = create_daily_aggregates(df)
        daily_df.to_csv("bangkok_weather_daily.csv", mode='w', header=True, index=True)
        print(f"Saved aggregated data for {len(daily_df)} days to daily CSV file")
    else:
        print("No data was collected")

def create_daily_aggregates(hourly_df):
    """Create daily aggregated statistics from hourly data"""
    # Group by date (ignoring time component)
    daily_groups = hourly_df.groupby(hourly_df.index.date)
    
    # Create daily aggregations
    daily_data = pd.DataFrame({
        'temperature_max': daily_groups['temp'].max(),
        'temperature_avg': daily_groups['temp'].mean(),
        'temperature_min': daily_groups['temp'].min(),
        'dew_point_max': daily_groups['dewPt'].max(),
        'dew_point_avg': daily_groups['dewPt'].mean(),
        'dew_point_min': daily_groups['dewPt'].min(),
        'humidity_max': daily_groups['rh'].max(),
        'humidity_avg': daily_groups['rh'].mean(),
        'humidity_min': daily_groups['rh'].min(),
        'wind_speed_max': daily_groups['wspd'].max(),
        'wind_speed_avg': daily_groups['wspd'].mean(),
        'wind_speed_min': daily_groups['wspd'].min(),
        'pressure_max': daily_groups['pressure'].max(),
        'pressure_avg': daily_groups['pressure'].mean(),
        'pressure_min': daily_groups['pressure'].min(),
        'precipitation': daily_groups['precip_total'].max(),  # Take the max as it's cumulative
    })
    
    # Convert index to proper datetime
    daily_data.index = pd.to_datetime(daily_data.index)
    
    return daily_data

if __name__ == "__main__":
    process_historical_data(2025, 2025)

In [13]:
weather_df = pd.read_csv("../../data/a1/bangkok_weather.csv", index_col=0, parse_dates=True)  

In [ ]:
weather_df.info()